## Mohammed almotiri 
### content based Movie recommender
- The following is a contnet based movie recommender using bert and spcay
- dataset : https://www.kaggle.com/datasets/yasirabdaali/tmdb-20000-trending-movies-dataset

## imports:

In [1]:
import pandas as pd 
import numpy as np
import spacy
import json
from sklearn.preprocessing import LabelEncoder
from sentence_transformers import SentenceTransformer
from rake_nltk import Rake
from transformers import BertTokenizer, BertModel

## functions

In [2]:
def clean_overview(text):
    cleaned_text = []
    for line in text:
#         print(line)
        doc = nlp(line)
        cleaned_text.append(" ".join([token.text.lower() for token in doc if not token.is_stop and not token.is_punct]))
    
    return cleaned_text

In [3]:
def keyword_extract(text):
    rake = Rake()
    list_keywords_text = []
    for line in text:
        list_keywords = ''
        rake.extract_keywords_from_text(line)
    
        for rating, keyword in rake.get_ranked_phrases_with_scores():
            if rating >1:
                list_keywords += keyword +' '
        list_keywords_text.append(list_keywords)
        del list_keywords
    return list_keywords_text

In [5]:
def genre_to_string2(df_genres):
    genre_as_string = list()
    genre_as_string2 = list()
    for genre in df_genres:
        
        try:
#             genre_as_string.append(",".join([i['name'] for i in json.loads(genre.replace("'", '"'))]))
            genre_as_string.append([i['name'] for i in json.loads(genre)])
          
        except:
            print(genre.replace('', ""))
    return genre_as_string

In [6]:
def unique_genres(genres):
    unique_genres = set()
    for genres in genres:
        unique_genres.update(genres.split(','))
    return list(unique_genres)

In [7]:
def genres_encoding(unique_genres, df_genre):
    genre_encoding = {}
    for genre in unique_genres:
        genre_encoding[genre] = df_genre.apply(lambda x: int(genre in x))
    return genre_encoding
#     for genre in unique_genres:
#         df[genre] = genre_encoding[genre]

In [8]:
def get_index(movie_name):
    index = df_movie.loc[df_movie.title == movie_name].index.values
#     print(index[0])
    try:
        
        if type(index[0]):
            return index[0]
    except:
        print('none')

In [9]:
def get_movie_vector(index_title,vector_space):
    
    return vector_space[index_title]

## data cleaning and prep

In [10]:
dict_genre_ids = {28: 'Action', 12: 'Adventure', 16: 'Animation', 35 : 'Comedy', 99 : 'Documentary', 18 : 'Drama', 10751: 
                 'Family', 14: 'Fantasy', 36: 'History' , 27 : 'Horror', 10402 : 'Music', 9648 : 'Mystery' , 10749 :'Romance',
                 878 : 'Science Fiction', 10770 : 'TV Movie', 53: 'Thriller', 10752 : 'War', 37 : 'Western', 80: 'Crime'}

In [75]:
df_movie2 = pd.read_csv('tmdb_5000_movies.csv')
df_movie2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [11]:
df_movie = pd.read_csv('new_movie2.csv')
df_movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18200 entries, 0 to 18199
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         18200 non-null  int64  
 1   id                 18200 non-null  int64  
 2   original_language  18200 non-null  object 
 3   original_title     18200 non-null  object 
 4   overview           18200 non-null  object 
 5   popularity         18200 non-null  float64
 6   release_date       18200 non-null  object 
 7   title              18200 non-null  object 
 8   vote_average       18200 non-null  float64
 9   vote_count         18200 non-null  float64
 10  string_genre       18163 non-null  object 
dtypes: float64(3), int64(2), object(6)
memory usage: 1.5+ MB


In [222]:
df_movie = df_movie.drop_duplicates(subset=['title'], keep='first')
df_movie

,Unnamed: 0,id,original_language,original_title,overview,popularity,release_date,title,vote_average,vote_count,...,Comedy,Romance,Family,Animation,Science Fiction,Documentary,Action,War,Mystery,clean_overview
0,0,19995,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,2009-12-10,avatar,7.2,11800.0,...,0,0,0,0,1,0,1,0,0,22nd century paraplegic marine dispatched moon...
1,1,285,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,2007-05-19,pirates of the caribbean: at world's end,6.9,4500.0,...,0,0,0,0,0,0,1,0,0,captain barbossa long believed dead come life ...
2,2,206647,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,2015-10-26,spectre,6.3,4466.0,...,0,0,0,0,0,0,1,0,0,cryptic message bond past sends trail uncover ...
3,3,49026,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,2012-07-16,the dark knight rises,7.6,9106.0,...,0,0,0,0,0,0,1,0,0,following death district attorney harvey dent ...
4,4,49529,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,2012-03-07,john carter,6.1,2124.0,...,0,0,0,0,1,0,1,0,0,john carter war weary military captain inexpli...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18195,45373,426272,en,Take Me,Ray is a fledgling entrepreneur who specialize...,9.222308,2017-05-05,take me,6.0,38.0,...,1,0,0,0,0,0,0,0,0,ray fledgling entrepreneur specializes high en...
18196,45380,432789,en,The Incredible Jessica James,"Burned by a bad breakup, a struggling New York...",5.667067,2017-01-27,the incredible jessica james,6.2,37.0,...,1,1,0,0,0,0,0,0,0,burned bad breakup struggling new york city pl...
18197,45437,455661,en,In a Heartbeat,A closeted boy runs the risk of being outed by...,20.821780,2017-06-01,in a heartbeat,8.3,146.0,...,1,1,1,1,0,0,0,0,0,closeted boy runs risk outed heart pops chest ...
18198,45441,14008,en,Cadet Kelly,Hyperactive teenager Kelly is enrolled into a ...,4.392389,2002-03-07,cadet kelly,5.2,145.0,...,1,0,0,0,0,0,0,0,0,hyperactive teenager kelly enrolled military s...


In [13]:
df_movie.title = df_movie.title.apply(lambda x: x.lower())

In [14]:
df_movie = df_movie.dropna()

for_enoding_genres = unique_genres(df_movie['string_genre'])
for_enoding_genres.remove('')

encoded_genres = genres_encoding(for_enoding_genres, df_movie['string_genre'])


for genre in for_enoding_genres:
    
    df_movie[genre] = encoded_genres[genre]
    
df_movie

,Unnamed: 0,id,original_language,original_title,overview,popularity,release_date,title,vote_average,vote_count,...,War,Comedy,Action,Adventure,History,Horror,Animation,Crime,Science Fiction,Mystery
0,0,19995,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,2009-12-10,avatar,7.2,11800.0,...,0,0,1,1,0,0,0,0,1,0
1,1,285,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,2007-05-19,pirates of the caribbean: at world's end,6.9,4500.0,...,0,0,1,1,0,0,0,0,0,0
2,2,206647,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,2015-10-26,spectre,6.3,4466.0,...,0,0,1,1,0,0,0,1,0,0
3,3,49026,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,2012-07-16,the dark knight rises,7.6,9106.0,...,0,0,1,0,0,0,0,1,0,0
4,4,49529,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,2012-03-07,john carter,6.1,2124.0,...,0,0,1,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18195,45373,426272,en,Take Me,Ray is a fledgling entrepreneur who specialize...,9.222308,2017-05-05,take me,6.0,38.0,...,0,1,0,0,0,0,0,1,0,0
18196,45380,432789,en,The Incredible Jessica James,"Burned by a bad breakup, a struggling New York...",5.667067,2017-01-27,the incredible jessica james,6.2,37.0,...,0,1,0,0,0,0,0,0,0,0
18197,45437,455661,en,In a Heartbeat,A closeted boy runs the risk of being outed by...,20.821780,2017-06-01,in a heartbeat,8.3,146.0,...,0,1,0,0,0,0,1,0,0,0
18198,45441,14008,en,Cadet Kelly,Hyperactive teenager Kelly is enrolled into a ...,4.392389,2002-03-07,cadet kelly,5.2,145.0,...,0,1,0,0,0,0,0,0,0,0


In [78]:
# df_movie2 = df_movie2.drop(columns=['homepage', 'tagline', 'belongs_to_collection'])
# df_movie2
df_movie2 = df_movie2.dropna()
df_movie2 = df_movie2.loc[df_movie2.vote_average >0]
df_movie2 = df_movie2.loc[df_movie2.vote_count >25]
df_movie2

,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4783,0,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 27, ""n...",226458,"[{""id"": 9712, ""name"": ""possession""}]",en,Backmask,"During an all-night, drug-fueled party at an a...",3.619167,"[{""name"": ""GO Productions"", ""id"": 2943}, {""nam...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2015-01-16,0,91.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Backmask,4.7,79
4788,12000,"[{""id"": 27, ""name"": ""Horror""}, {""id"": 35, ""nam...",692,"[{""id"": 237, ""name"": ""gay""}, {""id"": 900, ""name...",en,Pink Flamingos,Notorious Baltimore criminal and underground f...,4.553644,"[{""name"": ""Dreamland Productions"", ""id"": 407}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1972-03-12,6000000,93.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Pink Flamingos,6.2,110
4792,20000,"[{""id"": 80, ""name"": ""Crime""}, {""id"": 27, ""name...",36095,"[{""id"": 233, ""name"": ""japan""}, {""id"": 549, ""na...",ja,キュア,A wave of gruesome murders is sweeping Tokyo. ...,0.212443,"[{""name"": ""Daiei Studios"", ""id"": 881}]","[{""iso_3166_1"": ""JP"", ""name"": ""Japan""}]",1997-11-06,99000,111.0,"[{""iso_639_1"": ""ja"", ""name"": ""\u65e5\u672c\u8a...",Released,Cure,7.4,63
4796,7000,"[{""id"": 878, ""name"": ""Science Fiction""}, {""id""...",14337,"[{""id"": 1448, ""name"": ""distrust""}, {""id"": 2101...",en,Primer,Friends/fledgling entrepreneurs invent a d

In [554]:
genre_as_string = ''
genre_as_string2 = list()
for genres in df_movie2.genre_ids.to_list():
    genres = list(genres.replace('[', '').replace(']', '').split(','))

    try:
        genres = list(map(int, genres))
    except:
        genres = [0]
    for genre in genres:
        if genre == 0:
            genre_as_string = None
        else:
            genre_as_string = ",".join([dict_genre_ids[genre], genre_as_string])

    genre_as_string2.append(genre_as_string)
    genre_as_string = ''

In [555]:
df_movie2['string_genre'] = genre_as_string2
df_movie2['string_genre']
df_movie2 = df_movie2.dropna()
df_movie2

,id,title,original_language,original_title,overview,genre_ids,popularity,release_date,vote_average,vote_count,string_genre
0,766507,Prey,en,Prey,"When danger threatens her camp, the fierce and...","[28, 878, 53]",8530.890,2022-08-02,8.086,2828.0,"Thriller,Science Fiction,Action,"
1,361743,Top Gun: Maverick,en,Top Gun: Maverick,After more than thirty years of service as one...,"[28, 18]",3887.230,2022-05-24,8.331,2314.0,"Drama,Action,"
2,755566,Day Shift,en,Day Shift,An LA vampire hunter has a week to come up wit...,"[28, 14, 27]",2624.719,2022-08-10,6.777,455.0,"Horror,Fantasy,Action,"
3,438148,Minions: The Rise of Gru,en,Minions: The Rise of Gru,A fanboy of a supervillain supergroup known as...,"[16, 12, 35, 14]",4064.409,2022-06-29,7.772,1504.0,"Fantasy,Comedy,Adventure,Animation,"
4,614934,Elvis,en,Elvis,The life story of Elvis Presley as seen throug...,"[10402, 18, 36]",907.927,2022-06-22,7.918,1109.0,"History,Drama,Music,"
...,...,...,...,...,...,...,...,...,...,...,...
67782,8767,Stuntwoman,fr,L'Animal,Mike (Jean-Paul Belmondo) is a stuntman who wo...,[35],5.802,1977-10-05,6.375,68.0,"Comedy,"
67783,4296,Millennium,en,Millennium,An investigator seeking the cause of an airlin...,[878],7.342,1989-08-25,5.400,94.0,"Science Fiction,"
67791,858717,Harry and Meghan: Escaping the Palace,en,Harry and Meghan: Escaping the Palace,The Duke and Duchess of Sussex are taking on t...,"[10770, 18]",6.889,2021-09-06,6.800,143.0,"Drama,TV Movie,"
67812,127373,What Maisie Knew,en,What Maisie Knew,"The story frames on 7-year-old Maisie, caught ...",[18],7.065,2013-05-02,7.084,362.0,"Drama,"


In [76]:
df_movie2 = df_movie2.drop(columns=['homepage', 'tagline'])

In [16]:
df_movie = df_movie.drop_duplicates(subset=['title'], keep='first')

df_movie = df_movie.dropna()
df_movie.info()
# df_movie2 = df_movie2.drop_duplicates(subset=['title'], keep='first')

# df_movie2 = df_movie2.dropna()
# df_movie2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18101 entries, 0 to 18199
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         18101 non-null  int64  
 1   id                 18101 non-null  int64  
 2   original_language  18101 non-null  object 
 3   original_title     18101 non-null  object 
 4   overview           18101 non-null  object 
 5   popularity         18101 non-null  float64
 6   release_date       18101 non-null  object 
 7   title              18101 non-null  object 
 8   vote_average       18101 non-null  float64
 9   vote_count         18101 non-null  float64
 10  string_genre       18101 non-null  object 
 11  Romance            18101 non-null  int64  
 12  Thriller           18101 non-null  int64  
 13  Music              18101 non-null  int64  
 14  Family             18101 non-null  int64  
 15  Foreign            18101 non-null  int64  
 16  TV Movie           181

In [596]:
result = pd.concat([df_movie, df_movie2], axis=0, join='inner')
result = result.drop_duplicates(subset=['title'], keep='first')
result.to_csv('new_movie2.csv')

In [10]:
spacy.cli.download("en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [130]:
nlp = spacy.load("en_core_web_sm")

In [131]:
df_movie['clean_overview'] = clean_overview(df_movie.overview.to_list())

df_movie['clean_overview']

0        22nd century paraplegic marine dispatched moon...
1        captain barbossa long believed dead come life ...
2        cryptic message bond past sends trail uncover ...
3        following death district attorney harvey dent ...
4        john carter war weary military captain inexpli...
                               ...                        
18195    ray fledgling entrepreneur specializes high en...
18196    burned bad breakup struggling new york city pl...
18197    closeted boy runs risk outed heart pops chest ...
18198    hyperactive teenager kelly enrolled military s...
18199    chemist laboratory places table head alive fix...
Name: clean_overview, Length: 18163, dtype: object

In [43]:
df_movie['over_key'] = keyword_extract(df_movie.overview.to_list())
df_movie['over_key']

0        unique mission paraplegic marine moon pandora ...
1        long believed elizabeth swann come back captai...
2        terrible truth behind spectre secret service a...
3        gotham city police department district attorne...
4        reluctantly becomes embroiled former military ...
                               ...                        
18195    mysterious client contracts end simulated abdu...
18196    struggling new york city playwright makes divo...
18197                               closeted boy runs boy 
18198    new stepfather becomes hyperactive teenager ke...
18199    becomes truly colossal laboratory places upon ...
Name: over_key, Length: 18200, dtype: object

In [94]:
df_movie2['genres'].to_list()
df_movie2['string_genre'] = genre_to_string(df_movie2['genres'].to_list())
df_movie2['string_genre']

0       Action,Adventure,Fantasy,Science Fiction
1                       Adventure,Fantasy,Action
2                         Action,Adventure,Crime
3                    Action,Crime,Drama,Thriller
4               Action,Adventure,Science Fiction
                          ...                   
4783                             Thriller,Horror
4788                         Horror,Comedy,Crime
4792               Crime,Horror,Mystery,Thriller
4796              Science Fiction,Drama,Thriller
4798                       Action,Crime,Thriller
Name: string_genre, Length: 4020, dtype: object

In [131]:
# df_movie2['keywords'][1]
df_movie2['string_keywords'] = genre_to_string2(df_movie2['keywords'].to_list())
df_movie2['string_keywords'].to_list()[0]

['culture clash',
 'future',
 'space war',
 'space colony',
 'society',
 'space travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love affair',
 'anti war',
 'power relations',
 'mind and soul',
 '3d']

In [137]:
data = []
for text, keywords in zip(df_movie2['clean_overview'], df_movie2['string_keywords']):
    data.append({"text": text, "keywords": keywords})
data

[{'text': '22nd century paraplegic marine dispatched moon pandora unique mission torn following orders protecting alien civilization',
  'keywords': ['culture clash',
   'future',
   'space war',
   'space colony',
   'society',
   'space travel',
   'futuristic',
   'romance',
   'space',
   'alien',
   'tribe',
   'alien planet',
   'cgi',
   'marine',
   'soldier',
   'battle',
   'love affair',
   'anti war',
   'power relations',
   'mind and soul',
   '3d']},
 {'text': 'captain barbossa long believed dead come life headed edge earth turner elizabeth swann',
  'keywords': ['ocean',
   'drug abuse',
   'exotic island',
   'east india trading company',
   "love of one's life",
   'traitor',
   'shipwreck',
   'strong woman',
   'ship',
   'alliance',
   'calypso',
   'afterlife',
   'fighter',
   'pirate',
   'swashbuckler',
   'aftercreditsstinger']},
 {'text': 'cryptic message bond past sends trail uncover sinister organization m battles political forces secret service alive bond 

In [104]:
df_movie2['string_keywords'].to_list()

['culture clash,future,space war,space colony,society,space travel,futuristic,romance,space,alien,tribe,alien planet,cgi,marine,soldier,battle,love affair,anti war,power relations,mind and soul,3d',
 "ocean,drug abuse,exotic island,east india trading company,love of one's life,traitor,shipwreck,strong woman,ship,alliance,calypso,afterlife,fighter,pirate,swashbuckler,aftercreditsstinger",
 'spy,based on novel,secret agent,sequel,mi6,british secret service,united kingdom',
 'dc comics,crime fighter,terrorist,secret identity,burglar,hostage drama,time bomb,gotham city,vigilante,cover-up,superhero,villainess,tragic hero,terrorism,destruction,catwoman,cat burglar,imax,flood,criminal underworld,batman',
 'based on novel,mars,medallion,space travel,princess,alien,steampunk,martian,escape,edgar rice burroughs,alien race,superhuman strength,mars civilization,sword and planet,19th century,3d',
 "dual identity,amnesia,sandstorm,love of one's life,forgiveness,spider,wretch,death of a friend,egoman

In [224]:
df_movie.title = df_movie.title.apply(lambda x: x.lower())


In [56]:
df_movie = df_movie.dropna()

In [17]:
for_enoding_genres = unique_genres(df_movie['string_genre'])
for_enoding_genres.remove('')

In [18]:
for_enoding_genres

['Romance',
 'Thriller',
 'Music',
 'Family',
 'Foreign',
 'TV Movie',
 'Documentary',
 'Western',
 'Drama',
 'Fantasy',
 'War',
 'Comedy',
 'Action',
 'Adventure',
 'History',
 'Horror',
 'Animation',
 'Crime',
 'Science Fiction',
 'Mystery']

In [227]:
encoded_genres = genres_encoding(for_enoding_genres, df_movie['string_genre'])


for genre in for_enoding_genres:
    
    df_movie[genre] = encoded_genres[genre]
    
df_movie

,Unnamed: 0,id,original_language,original_title,overview,popularity,release_date,title,vote_average,vote_count,...,Comedy,Romance,Family,Animation,Science Fiction,Documentary,Action,War,Mystery,clean_overview
0,0,19995,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,2009-12-10,avatar,7.2,11800.0,...,0,0,0,0,1,0,1,0,0,22nd century paraplegic marine dispatched moon...
1,1,285,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,2007-05-19,pirates of the caribbean: at world's end,6.9,4500.0,...,0,0,0,0,0,0,1,0,0,captain barbossa long believed dead come life ...
2,2,206647,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,2015-10-26,spectre,6.3,4466.0,...,0,0,0,0,0,0,1,0,0,cryptic message bond past sends trail uncover ...
3,3,49026,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,2012-07-16,the dark knight rises,7.6,9106.0,...,0,0,0,0,0,0,1,0,0,following death district attorney harvey dent ...
4,4,49529,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,2012-03-07,john carter,6.1,2124.0,...,0,0,0,0,1,0,1,0,0,john carter war weary military captain inexpli...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18195,45373,426272,en,Take Me,Ray is a fledgling entrepreneur who specialize...,9.222308,2017-05-05,take me,6.0,38.0,...,1,0,0,0,0,0,0,0,0,ray fledgling entrepreneur specializes high en...
18196,45380,432789,en,The Incredible Jessica James,"Burned by a bad breakup, a struggling New York...",5.667067,2017-01-27,the incredible jessica james,6.2,37.0,...,1,1,0,0,0,0,0,0,0,burned bad breakup struggling new york city pl...
18197,45437,455661,en,In a Heartbeat,A closeted boy runs the risk of being outed by...,20.821780,2017-06-01,in a heartbeat,8.3,146.0,...,1,1,1,1,0,0,0,0,0,closeted boy runs risk outed heart pops chest ...
18198,45441,14008,en,Cadet Kelly,Hyperactive teenager Kelly is enrolled into a ...,4.392389,2002-03-07,cadet kelly,5.2,145.0,...,1,0,0,0,0,0,0,0,0,hyperactive teenager kelly enrolled military s...


## content encoding

In [23]:
X = np.array(df_movie.overview)

text_data = X
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
embeddings2 = model.encode(text_data, show_progress_bar=True)

Batches:   0%|          | 0/566 [00:00<?, ?it/s]

In [66]:
len(embeddings[0])

768

In [19]:
df_movie2 = df_movie.iloc[:, 20:]
# df_movie3 =df_movie2.copy()
# # df_movie3['overview'] = df_movie['clean_overview']
# # df_movie3.to_csv('final.csv')
# df_movie2["vote_average"] = df_movie["vote_average"] // 10

In [662]:
del combined_embeddings

In [233]:
df_movie2 = df_movie.iloc[:, 11:30]
df_movie2.head(2)

,Horror,Crime,Foreign,TV Movie,Drama,Thriller,Western,Adventure,Fantasy,Music,History,Comedy,Romance,Family,Animation,Science Fiction,Documentary,Action,War
0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,1,0
1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0


In [20]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(df_movie2)
normalized_combined_embeddings = scaler.transform(df_movie2)

In [ ]:
combined_embeddings = np.hstack((np.array(embeddings2),  normalized_combined_embeddings))
combined_embeddings

In [24]:
normalized_combined_embeddings


combined_embeddings = np.hstack((np.array(embeddings2), normalized_combined_embeddings))
combined_embeddings

array([[-0.71422577, -0.60569674,  0.3367987 , ..., -0.38540966,
         2.92523882, -0.285362  ],
       [-0.65981573, -0.6304599 ,  0.82535315, ..., -0.38540966,
        -0.34185243, -0.285362  ],
       [-0.48309135, -0.48895025,  0.30447885, ...,  2.59464174,
        -0.34185243, -0.285362  ],
       ...,
       [-0.9471004 , -0.55319548,  0.30680767, ..., -0.38540966,
        -0.34185243, -0.285362  ],
       [-0.53153509, -0.3196868 , -0.11739904, ..., -0.38540966,
        -0.34185243, -0.285362  ],
       [-0.24550918,  0.24456602,  0.52773261, ..., -0.38540966,
         2.92523882, -0.285362  ]])

## model fiting and testing

In [25]:
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
# Create a k-NN model
knn = NearestNeighbors(n_neighbors=5)
knn.fit(combined_embeddings)


# kmeans = KMeans(n_clusters=100, random_state=0)

# kmeans.fit(normalized_combined_embeddings)

# cluster_labels = kmeans.labels_

NearestNeighbors()

In [ ]:
df_movie.iloc[index]['overview']

In [650]:
df_movie2.iloc[index:]

,Horror,Crime,Foreign,TV Movie,Drama,Thriller,Western,Adventure,Fantasy,Music,History,Comedy,Romance,Family,Animation,Science Fiction,Documentary,Action,War
1394,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1395,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
1396,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1397,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
1398,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18195,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
18196,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
18197,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0
18198,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [27]:
index = get_index('O Brother, Where Art Thou?'.lower())
print(index)

1790


In [28]:
distances, indices = knn.kneighbors(get_movie_vector(index,combined_embeddings).reshape(1, -1), n_neighbors=11)

# Recommend movies based on the nearest neighbors
list_rec_mov = []
for i in indices[0]:
    print('\t',df_movie.string_genre[i],'\t',df_movie.title[i],'\t',df_movie.overview[i],'\n\n\n\n\n')

# liked_movie_cluster = cluster_labels[index]
# print(cluster_labels, liked_movie_cluster)
# # Find all the movies in the same cluster
# movies_in_cluster = [i for i, label in enumerate(cluster_labels) if label == liked_movie_cluster]

# # Recommend movies from the same cluster, excluding the liked movie
# recommended_movies = [df_movie.title[i] for i in movies_in_cluster if i != index]
# recommended_movies

	 Action,Adventure,Comedy 	 o brother, where art thou? 	 In the deep south during the 1930s, three escaped convicts search for hidden treasure while a relentless lawman pursues them. On their journey they come across many comical characters and incredible situations. Based upon Homer's 'Odyssey'. 





	 Action,Adventure,Comedy 	 the dukes of hazzard 	 Cousins, Bo and Luke Duke, with the help of their eye-catching cousin, Daisy and moonshine-running Uncle Jesse, try and save the family farm from being destroyed by Hazzard County's corrupt commissioner, Boss Hogg. Their efforts constantly find the 'Duke Boys' eluding authorities in 'The General Lee', their 1969 orange Dodge Charger that keeps them one step ahead of the dimwitted antics of the small southern town's Sheriff, Roscoe P. Coltrane. 





	 Drama,Romance 	 à nos amours 	 A portrait of youth in bloom; a tale of one family's dissolution; a reflection upon the danger and the mystery in living.  Sandrine Bonnaire plays Suzanne, a 

In [662]:
# cos_sim_data = pd.DataFrame(cosine_similarity(X))
from sklearn.metrics.pairwise import cosine_similarity

In [547]:
cos_sim_data = pd.DataFrame(cosine_similarity(combined_embeddings))
def give_recommendations(index,print_recommendation = False,print_recommendation_plots= False,print_genres =False):
    index_recomm =cos_sim_data.loc[index].sort_values(ascending=False).index.tolist()[1:6]
    movies_recomm =  df_movie['title'].loc[index_recomm].values
    result = {'Movies':movies_recomm,'Index':index_recomm}
    if print_recommendation==True:
        print('The watched movie is this one: %s \n'%(df_movie['title'].loc[index]))
        k=1
        for movie in movies_recomm:
            print('The number %i recommended movie is this one: %s \n'%(k,movie))
    if print_recommendation_plots==True:
        print('The plot of the watched movie is this one:\n %s \n'%(df_movie['overview'].loc[index]))
        k=1
        for q in range(len(movies_recomm)):
            plot_q = df_movie['overview'].loc[index_recomm[q]]
            print('The plot of the number %i recommended movie is this one:\n %s \n'%(k,plot_q))
            k=k+1
    if print_genres==True:
        print('The genres of the watched movie is this one:\n %s \n'%(df_movie.string_genre.loc[index]))
        k=1
        for q in range(len(movies_recomm)):
            plot_q = df_movie.string_genre.loc[index_recomm[q]]
            print('The plot of the number %i recommended movie is this one:\n %s \n'%(k,plot_q))
            k=k+1
    return result

In [640]:
give_recommendations(index = get_index('Collateral'.lower()),print_recommendation = True, print_recommendation_plots= True,print_genres =True)

The watched movie is this one: collateral 

The number 1 recommended movie is this one: a dog named palma 

The number 1 recommended movie is this one: those who wish me dead 

The number 1 recommended movie is this one: spinning man 

The number 1 recommended movie is this one: the trust 

The number 1 recommended movie is this one: barefoot 

The plot of the watched movie is this one:
 Cab driver Max picks up a man who offers him $600 to drive him around. But the promise of easy money sours when Max realizes his fare is an assassin. 

The plot of the number 1 recommended movie is this one:
 The film is inspired by an incredible story of a shepherd dog named Palm who was inadvertently left in the airport by her owner. She befriends nine-year old Nicholas whose mother dies leaving him with a father he barely knows - a pilot who finds the dog at the airport. It's a story of amazing adventures, true friendship and unconditional love. 

The plot of the number 2 recommended movie is this o

{'Movies': array(['a dog named palma', 'those who wish me dead', 'spinning man',
        'the trust', 'barefoot'], dtype=object),
 'Index': [5632, 5364, 8554, 9333, 11432]}